# Mixed problema: Darcy's flow in porous media
---

In [1]:
#try:
#  import gmsh
#except ImportError:
#  !wget "https://github.com/fem-on-colab/fem-on-colab.github.io/raw/7f220b6/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh" && bash "/tmp/gmsh-install.sh"
#  import gmsh

#try:
#  import dolfinx
#except ImportError:
#  !wget "https://github.com/fem-on-colab/fem-on-colab.github.io/raw/7f220b6/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
#  import dolfinx


try:
    import dolfinx
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx

--2022-12-27 03:32:11--  https://fem-on-colab.github.io/releases/fenicsx-install-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.109.153, 185.199.110.153, 185.199.108.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4252 (4.2K) [application/x-sh]
Saving to: ‘/tmp/fenicsx-install.sh’

/tmp/fenicsx-instal 100%[===================>]   4.15K  --.-KB/s    in 0s      

2022-12-27 03:32:11 (25.4 MB/s) - ‘/tmp/fenicsx-install.sh’ saved [4252/4252]

+ INSTALL_PREFIX=/usr/local
++ awk -F/ '{print NF-1}'
++ echo /usr/local
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FENICSX_INSTALLED=/usr/local/share/fem-on-colab/fenicsx.installed
+ [[ ! -f /usr/local/share/fem-on-colab/fenicsx.installed ]]
+ PYBIND11_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/9313046/releases/pybind1

In [4]:
from dolfinx import mesh, fem, io, plot

from ufl import SpatialCoordinate, FiniteElement, MixedElement, TestFunction, TrialFunction, split, Measure, dx, ds, grad, div, dot, inner, as_vector, FacetNormal

import numpy as np
from mpi4py import MPI
from petsc4py.PETSc import ScalarType

In [20]:
# Computational mesh 
Lx, Ly = 2.0, 1.0
#cell_type = mesh.CellType.triangle
cell_type = mesh.CellType.quadrilateral
msh = mesh.create_rectangle(comm=MPI.COMM_WORLD,
                            points=((0.0, 0.0), (Lx, Ly)), n=(64, 32),
                            cell_type=cell_type)

boundaries = [(1, lambda x: np.isclose(x[0], 0)),
              (2, lambda x: np.isclose(x[0], Lx)),
              (3, lambda x: np.isclose(x[1], 0)),
              (4, lambda x: np.isclose(x[1], Ly))]

facet_indices, facet_markers = [], []
fdim = msh.topology.dim - 1
for (marker, locator) in boundaries:
    facets = mesh.locate_entities(msh, fdim, locator)
    facet_indices.append(facets)
    facet_markers.append(np.full_like(facets, marker))
facet_indices = np.hstack(facet_indices).astype(np.int32)
facet_markers = np.hstack(facet_markers).astype(np.int32)
sorted_facets = np.argsort(facet_indices)
facet_tag = mesh.meshtags(msh, fdim, facet_indices[sorted_facets], facet_markers[sorted_facets])

ds = Measure("ds", domain=msh, subdomain_data=facet_tag)

one = fem.Constant(msh, 1.0)
for i in [1, 2, 3, 4]:
  length_form = fem.form( one*ds(i) )
  lengthside = fem.assemble_scalar(length_form)
  print(lengthside)


1.0
1.0
2.0
2.0


In [21]:
degree_u = 2
degree_p = 1

if(cell_type == mesh.CellType.triangle):
    print(msh.ufl_cell())
    el_u = FiniteElement("RT", msh.ufl_cell(), degree_u)
    el_p = FiniteElement("DG",  msh.ufl_cell(), degree_p)
else:
    print(msh.ufl_cell())
    el_u = FiniteElement("RTCF", msh.ufl_cell(), degree_u)
    el_p = FiniteElement("DQ",  msh.ufl_cell(), degree_p)
    
el_m  = MixedElement([el_u , el_p])
W = fem.FunctionSpace(msh, el_m)

# Define trial and test functions
#(u, p) = TrialFunctions(W)
#(v, q) = TestFunctions(W)

TstF = TrialFunction(W)
TriF = TestFunction(W)
(u, p) = split(TstF)
(v, q) = split(TriF)

# Define source function
x = SpatialCoordinate(msh)

f = fem.Constant(msh, 0.0)

pleft  = fem.Constant(msh, 100.0)
pright = fem.Constant(msh, 0.0)

# Define variational form
n = FacetNormal(msh)
a = (inner(u, v) - div(v)*p - div(u)*q) * dx
L = f * q * dx - pleft*dot(v,n)*ds(1) - pright*dot(v,n)*ds(2)

def g(x, n):
    return (0.0*x[0]*n[0], 0.0*x[0]*n[1])

W0, U_to_W = W.sub(0).collapse()

w_bc = fem.Function(W0)
top_facets = mesh.locate_entities_boundary(msh, msh.topology.dim-1, lambda x: np.isclose(x[1], Ly))
top_cells = mesh.compute_incident_entities(msh, top_facets, msh.topology.dim-1, msh.topology.dim)
w_bc.interpolate(lambda x: g(x, (0,1)), top_cells)

bottom_facets = mesh.locate_entities_boundary(msh, msh.topology.dim-1, lambda x: np.isclose(x[1], 0))
bottom_cells = mesh.compute_incident_entities(msh, bottom_facets, msh.topology.dim-1, msh.topology.dim)
w_bc.interpolate(lambda x: g(x, (0,1)), bottom_cells)
w_bc.x.scatter_forward()

bc_dofs = fem.locate_dofs_topological((W.sub(0), W0), msh.topology.dim-1, np.hstack([top_facets, bottom_facets]))

bcs = [fem.dirichletbc(w_bc, bc_dofs, W.sub(0))]

wh = fem.Function(W)
#petsc_opts={"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps", "ksp_monitor": None}
petsc_opts={"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "superlu", "ksp_monitor": None}
problem = fem.petsc.LinearProblem(a, L, bcs=bcs, u=wh, petsc_options=petsc_opts)
problem.solve()
uh = wh.sub(0).collapse()
ph = wh.sub(1).collapse()
#(uh,ph) = wh.split()

inflow = fem.form(dot(uh,n)*ds(1))
outflow = fem.form(dot(uh,n)*ds(2))
bottomflow = fem.form(dot(uh,n)*ds(3))
topflow = fem.form(dot(uh,n)*ds(3))

print(fem.assemble_scalar(inflow), fem.assemble_scalar(outflow), fem.assemble_scalar(topflow), fem.assemble_scalar(bottomflow))

pex = lambda x: -(pleft - pright)*x[0]/Lx + pleft
ep = fem.form( ( (pex(x)-ph)**2 ) * dx)
print(fem.assemble_scalar(ep))

print(np.max(ph.x.array[:]), np.min(ph.x.array[:]))
print(np.max(uh.x.array[:]), np.min(uh.x.array[:]))

if(cell_type == mesh.CellType.triangle):
    BDM_out = fem.VectorFunctionSpace(msh, ("DG", 1))
else:
    BDM_out = fem.VectorFunctionSpace(msh, ("DQ", 1))
u_out = fem.Function(BDM_out)
u_out.interpolate(uh)
u_out.name = "Velocity"
uh.name = "Velocity"
with io.XDMFFile(msh.comm, "vel.xdmf", "w") as xdmf:
    xdmf.write_mesh(msh)
    xdmf.write_function(uh)

ph.name = "Pressure"
with io.XDMFFile(msh.comm, "pres.xdmf", "w") as xdmf:
    xdmf.write_mesh(msh)
    xdmf.write_function(ph)

quadrilateral
  Residual norms for dolfinx_solve_140623860513088 solve.
  0 KSP Residual norm 5.656854249492e+02 
  1 KSP Residual norm 2.020364704639e-12 
-50.00000000000208 49.99999999999941 0.0 0.0
1.4690462444979632e-25
100.00000000000007 -3.1086244689504383e-15
5.151434834260726e-13 -1.5625000000003126
